In [1]:
from transformers import AutoTokenizer
from pathlib import Path
import numpy as np
import torch
from transformers import AutoModelForTokenClassification
from tokenizers import AddedToken
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import pandas as pd
from seqeval.metrics import recall_score, precision_score, f1_score, accuracy_score

kaggle=False

path="/kaggle/input/pii-detection-removal-from-educational-data" if kaggle else "data"
train_path = path + "/train.json"
test_path = path + "/test.json"

model_path = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-base" if kaggle else "microsoft/deberta-v3-base"


if not kaggle: import neptune


In [2]:
cross_entropy_weight_multi = 400

CROSS_ENTROPY_WEIGHTS = [cross_entropy_weight_multi]*12
CROSS_ENTROPY_WEIGHTS.append(1)

parameter= {
    "model": model_path,
    "max_length": 1024,
    "inference_max_length": 2000,
    "batch_size": 9,
    "inference_batch_size": 3,
    "lr": 1e-3,
    "lr_scale_unfreeze": 0.1,
    "filter_no_pii_percent_allow": 0.2,
    "notebook": "20_deberta base_1024len.ipynb",
    "CROSS_ENTROPY_WEIGHT_MULTI": cross_entropy_weight_multi,
    "epochs_before_unfreeze": 4,
    "epochs_after_unfreeze": 6,
    "repeat_unfreeze_train_n_times": 2,
    "validate_every_n_epochs": 2,
    "train_test_split": 0.2,
    "num_proc": 16
}

print(parameter)

{'model': 'microsoft/deberta-v3-base', 'max_length': 1024, 'inference_max_length': 2000, 'batch_size': 9, 'inference_batch_size': 3, 'lr': 0.001, 'lr_scale_unfreeze': 0.1, 'filter_no_pii_percent_allow': 0.2, 'notebook': '20_deberta base_1024len.ipynb', 'CROSS_ENTROPY_WEIGHT_MULTI': 400, 'epochs_before_unfreeze': 4, 'epochs_after_unfreeze': 6, 'repeat_unfreeze_train_n_times': 2, 'validate_every_n_epochs': 2, 'train_test_split': 0.2, 'num_proc': 16}


In [3]:
target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
]

In [4]:
from itertools import chain
import json

data = json.load(open(train_path))
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

In [5]:
import random

def tokenize(example, tokenizer, label2id, max_length, all_labels_list):
    text = []
    import numpy as np

    # these are at the character level
    labels = []
    targets = []

    for t, l, ws in zip(example["tokens"], example["labels"], example["trailing_whitespace"]):

        text.append(t)
        labels.extend([l]*len(t))
        
        if l in all_labels_list:
            targets.append(1)
        else:
            targets.append(0)
        # if there is trailing whitespace
        if ws:
            text.append(" ")
            labels.append("O")

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=max_length, padding="max_length")
    
    target_num = sum(targets)
    labels = np.array(labels)

    text = "".join(text)
    token_labels = []

    for start_idx, end_idx in tokenized.offset_mapping:

        # CLS token
        if start_idx == 0 and end_idx == 0: 
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        try:
            token_labels.append(label2id[labels[start_idx]])
        except:
            token_labels.append(label2id["O"])

    length = len(tokenized.input_ids)

    return {
        **tokenized,
        "labels": token_labels,
        "length": length,
        "target_num": target_num,
        "group": 1 if target_num>0 else 0
    }

# https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/468844
def filter_no_pii(example, percent_allow=parameter["filter_no_pii_percent_allow"]):
    # Return True if there is PII
    # Or 20% of the time if there isn't
    has_pii = set("O") != set(example["labels"])
    return has_pii or (random.random() < percent_allow)

In [6]:
from datasets import Dataset
class PiiDataset(torch.utils.data.Dataset):
    def __init__(self, ds):
        self.dataset = ds
        
    def __getitem__(self, idx):
        vals=self.dataset[idx]
        input_ids = torch.tensor(vals["input_ids"])
        attention_mask = torch.tensor(vals["attention_mask"])
        labels = torch.tensor(vals["labels"], dtype=torch.long)
        return input_ids, attention_mask, labels
    
    def __len__(self):
        return len(self.dataset)

data = json.load(open(train_path))
print(data[0].keys())
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "labels": [x["labels"] for x in data],
})
    
tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id, "max_length": parameter["max_length"], "all_labels_list": target}, num_proc=parameter["num_proc"])

my_dataset=PiiDataset(ds)



dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map (num_proc=16):   0%|          | 0/6807 [00:00<?, ? examples/s]

In [7]:
loader=torch.utils.data.DataLoader(my_dataset, batch_size=8, shuffle=True)

for id, attention_mask, labels in loader:
    print(id.shape)
    print(attention_mask.shape)
    print(labels.shape)
    break

torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])


In [8]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class MyModel(torch.nn.Module):
    def __init__(self, model_name, num_labels, dropout_p=0.4):
        super().__init__()
        self.model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)
        self.softmax=torch.nn.Softmax(dim=-1)
        self.freeze()

    def freeze(self):
        for param in self.model.parameters():
            param.requires_grad = False
        for param in self.model.classifier.parameters():
            param.requires_grad = True
        for name, param in self.model.named_parameters():
            if param.requires_grad==True:
                print("still learning", name, "parameter_size:", param.size())

    def unfreeze(self):
        for param in self.model.parameters():
            param.requires_grad = True
        
    def forward(self, input_ids, attention_mask, labels=None):
        if labels is not None:
            out=self.model(input_ids, attention_mask=attention_mask, labels=labels)['logits']
        else:
            out=self.model(input_ids, attention_mask=attention_mask)['logits']
        out=self.softmax(out)
        return out

model = MyModel(parameter['model'], len(label2id))

model= model.to(device)
for id, attention_mask, labels in loader:
    print(id.shape)
    print(attention_mask.shape)
    print(labels.shape)
    print(labels)
    id = id.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)
    print(model(id, attention_mask, labels).shape)
    break

#free gpu memory
del model
torch.cuda.empty_cache()

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


still learning classifier.weight parameter_size: torch.Size([13, 768])
still learning classifier.bias parameter_size: torch.Size([13])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12],
        ...,
        [12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12]])
torch.Size([8, 1024, 13])


In [9]:
from torch.cuda.amp import GradScaler, autocast
class Learner():
    def __init__(self, model, train_dataloader, valid_dataloader, parameter=None):
        self.model=model
        #self.loss_fn=torch.nn.CrossEntropyLoss()
        self.loss_fn=torch.nn.CrossEntropyLoss(ignore_index=-100, weight=torch.tensor(CROSS_ENTROPY_WEIGHTS, dtype=torch.float32).to(device))
        self.device=torch.device("cpu")
        if torch.cuda.is_available():
            self.device=torch.device("cuda")
        self.model.to(self.device)
        self.parameter = parameter

        if not kaggle:
            self.run = neptune.init_run(
                project="bernd.heidemann/PII",
                api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzNjBlYzVkNi0zZTUwLTQ1ODYtODhlNC02NDUxNDg0MDdjNzUifQ==",
            )  # your credentials
            self.run["parameters"] = {
                **self.parameter
            }

        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.non_pii_label=label2id["O"]
        self.all_labels = target + ["O"]

    def fit(self, lr=0.1, epochs=10):
        scaler = GradScaler()
        optimizer=torch.optim.AdamW(self.model.parameters(), lr=lr)
        T_0 = epochs//3          # Number of epochs before the first restart
        T_mult = 2        # Factor by which T_0 is multiplied after each restart
        eta_min = lr*0.01   # Minimum learning rate at restarts

        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=T_0, T_mult=T_mult, eta_min=eta_min)
        bar = tqdm(total=len(self.train_dataloader) * epochs, desc="Training")
        bar.set_description("Epoch 0/{}".format(epochs))
        for epoch in range(epochs):
            self.model.train()     
            for ids, att_mask, labels in self.train_dataloader:
                optimizer.zero_grad()
                with autocast(dtype=torch.float16):
                    ids=ids.to(self.device)
                    labels=labels.to(self.device)
                    att_mask=att_mask.to(self.device)
                    pred=self.model(ids, att_mask, labels)
                    pred = pred.permute(0, 2, 1)
                    loss=self.loss_fn(pred, labels)
                    if not kaggle:
                        self.run["train_loss"].log(loss.item())
                    bar.update(1)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
               
            if not kaggle:
                self.run["learnrate"].log(optimizer.param_groups[0]["lr"])
            scheduler.step()
            self.model.eval()
            # log current state to neptune
            if self.valid_dataloader is not None and len(self.valid_dataloader)>0 and (epoch+1) % self.parameter["validate_every_n_epochs"] == 0:
                metrics=self.get_accuracy()
                if not kaggle:
                    self.run["valid_accuracy"].log(metrics["accuracy"])
                    self.run["valid_loss"].log(metrics["loss"])
                    self.run["valid_f1"].log(metrics["f1"])
                    self.run["valid_f5"].log(metrics["f5"])
                    self.run["valid_precision"].log(metrics["precision"])
                    self.run["valid_recall"].log(metrics["recall"])
                bar.set_description("Epoch {}/{} validAccuracy: {:.2f} validLoss: {:.2f} valid_f5: {:.2f}".format(epoch+1, epochs, metrics["accuracy"], metrics["loss"], metrics["f5"]))

    def get_accuracy(self):
        self.model.eval()
        with torch.no_grad():
            losses=[]
            batch_metrics=[]
            for ids, att_mask, labels in self.valid_dataloader:
                ids=ids.to(self.device)
                labels=labels.to(self.device)
                att_mask=att_mask.to(self.device)
                pred=self.model(ids, att_mask, labels)
                f_beta_score_results = self.compute_metrics(labels, pred)
                batch_metrics.append(f_beta_score_results)
                pred = pred.permute(0, 2, 1)
                loss=self.loss_fn(pred, labels)
                losses.append(loss.item())
            # calc mean of the dict entries in batch_metrics
            f1_scores = np.mean([x["f1"] for x in batch_metrics])
            recall_scores = np.mean([x["recall"] for x in batch_metrics])
            precision_scores = np.mean([x["precision"] for x in batch_metrics])
            accuracy_scores = np.mean([x["accuracy"] for x in batch_metrics])
            f5 = np.mean([x["f5"] for x in batch_metrics])
            return {
                "accuracy": accuracy_scores,
                "loss": np.mean(losses),
                "f1": f1_scores,
                "recall": recall_scores,
                "precision": precision_scores,
                "f5": f5
            }
        
    def compute_metrics(self, labels, predictions):
        predictions = torch.argmax(predictions, axis=2)
        true_predictions = []
        true_labels = []
        for pred, label in zip(predictions, labels):
            true_pred = []
            true_label = []
            for p, l in zip(pred, label):
                if l != -100:  ## skip zero
                    true_pred.append(self.all_labels[p]) # Add label
                    true_label.append(self.all_labels[l]) # Add label
            true_predictions.append(true_pred)
            true_labels.append(true_label)
        # Compute recall, precision and f1 score
        recall = recall_score(true_labels, true_predictions, zero_division=1e-7)
        precision = precision_score(true_labels, true_predictions, zero_division=1e-7)
        accuracy = accuracy_score(true_labels, true_predictions)
        f1 = f1_score(true_labels, true_predictions)
        # calculate f1 score from scratch, beta is 5
        f5 = (1 + 5**2) * (precision * recall) / ((5**2 * precision) + recall)


        result = {'f1': f1,
                    'recall': recall,
                    'precision': precision,
                    'accuracy': accuracy,
                    'f5': f5}
        return result
        

    

In [10]:
def tokenize_inference(example, tokenizer, max_length):
        text = []
        for t,  ws in zip(example["tokens"], example["trailing_whitespace"]):
            text.append(t)
            if ws:
                text.append(" ")
        tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=max_length, padding="max_length")
        text = "".join(text)
        length = len(tokenized.input_ids)
        return {
            **tokenized,
            "length": length,
        }
        
class TestTokenizer():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def preprocess(self, example):
        # Preprocess the tokens and labels by adding trailing whitespace and labels
        tokens = []
        tokens_without_ws = []
        token_map = [] # Use the index as labels
        index = 0
        for token, t_ws in zip(example["tokens"], example["trailing_whitespace"]):
            tokens_without_ws.append(token)
            tokens.append(token)
            token_map.extend([index] * len(token))
            # Added trailing whitespace and label if true and 
            if t_ws:
                tokens.append(" ")
                token_map.append(-1)
            index += 1
        return tokens, token_map, tokens_without_ws
    
    def tokenize(self, example):
        tokens, token_map, tokens_without_ws = self.preprocess(example)
        text = "".join(tokens)
        tokenized = self.tokenizer(text, return_offsets_mapping=True, padding="max_length",
                                   truncation=True, max_length=parameter["inference_max_length"])
        return {**tokenized, "token_map": token_map, "tokens": tokens, "tokens_without_ws": tokens_without_ws} 

class PiiDatasetInference(torch.utils.data.Dataset):
        def __init__(self, dataset, tokenizer):
            self.dataset = dataset
            self.tokenizer=TestTokenizer(tokenizer)
            
        def __getitem__(self, idx):
            vals=self.tokenizer.tokenize(self.dataset[idx])
            input_ids = torch.tensor(vals["input_ids"])
            attention_mask = torch.tensor(vals["attention_mask"])
            document_id = self.dataset[idx]["document"]
            return input_ids, attention_mask, document_id, vals
        
        def __len__(self):
            return len(self.dataset)

def inference(model):
    data = json.load(open(train_path))
    from itertools import chain
    all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
    label2id = {l: i for i,l in enumerate(all_labels)}
    id2label = {v:k for k,v in label2id.items()}

    tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
    data = json.load(open(test_path))
    my_dataset=PiiDatasetInference(data, tokenizer)
    loader=torch.utils.data.DataLoader(my_dataset, batch_size=parameter['batch_size'])
    for id, attention_mask, document_id, vals  in loader:
        id = id.to(device)
        print(id.shape)
        attention_mask = attention_mask.to(device)
        preds=model(id, attention_mask).argmax(dim=2)

        for pred, id in zip(preds.flatten(), id.flatten()):
            if pred != 12:
                print(f"TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
        print("next")

# Convert preds to a list of dictionaries
def to_test_submission(preds=None, dataset=None, document_ids=None, id2label=None):
    triplets = []
    row_id = 0
    results = []
    
    for i in range(len(preds)):
        input_ids, attention_mask, document_id, vals = dataset[i]
        token_map=vals["token_map"]
        offsets=vals["offset_mapping"]
        tokens=vals["tokens_without_ws"]
        #print("tokens", tokens)
        pred=preds[i]
        original_text = tokenizer.decode(input_ids)[6:] # skip CLS token
        #print("original_text", original_text)
        #print("token_map", token_map)
        #print("offsets", offsets)   
        #print("pred", pred)

        for token_pred, input_id, (start_idx, end_idx) in zip(pred, input_ids, offsets):
            #print("\nnow doing ", start_idx,  end_idx, token_pred)
            if start_idx == 0 and end_idx == 0: # Skip 0 offset
                continue
            # Skip spaces 
            while start_idx < len(token_map):
                #print("loop, start_idx now", start_idx) 
                #print(" tokens[token_map[start_idx]]: ", tokens[token_map[start_idx]] if not tokens[token_map[start_idx]].isspace() else "WHITESPACE")          
                if token_map[start_idx] == -1: # Skip unknown tokens               
                    start_idx += 1
                elif tokens[token_map[start_idx]].isspace(): # Skip white space
                    start_idx += 1
                else:
                    break
            # Ensure start index < length
            if start_idx < len(token_map):
                token_id = token_map[start_idx]
                #print("token_id", token_id)
                #token_id= input_id.item()
                label_pred = id2label[token_pred.item()]
                #print("label_pred", label_pred)
                # ignore "O" and whitespace preds
                if label_pred != "O" and token_id != -1:
                    #print("is PII", token_id, label_pred)
                    token_str = tokens[token_id]
                    triplet = (label_pred, token_id, token_str)
                    if triplet not in triplets:
                        results.append({
                            "row_id": row_id, 
                            "document": document_id,
                            "token": token_id, 
                            "label": label_pred,
                            "token_str": token_str
                        })
                        triplets.append(triplet)
                        row_id += 1

    # Create a dataframe 
    return results

def create_submission(model, filename="submission.csv"):
    data = json.load(open(train_path))
    from itertools import chain
    all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
    label2id = {l: i for i,l in enumerate(all_labels)}
    id2label = {v:k for k,v in label2id.items()}

    data=json.load(open(test_path))
    tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
    my_dataset=PiiDatasetInference(data, tokenizer)
    loader=torch.utils.data.DataLoader(my_dataset, batch_size=1, shuffle=False)

    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    

    # stack all predictions into tensor
    all_preds = []

    for id, attention_mask, document_ids, vals in loader:
        id=id.to(device)
        attention_mask=attention_mask.to(device)
        preds=model(id, attention_mask).argmax(dim=2)
        all_preds.append(preds)
        #for pred, id in zip(preds.flatten(), id.flatten()):
        #    if pred != 12:
                #print(f"Document: {document_id.item()} TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
        #        output[row_id]={"document":document_id.item(), "token":id.item(), "label":id2label[pred.item()]}
        #        row_id+=1
        #for pred, id in zip(preds.flatten(), id.flatten()):
        #    if pred != 12:
        #        print(f"TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
    
   
    all_preds = torch.cat(all_preds, dim=0)
    
    results = to_test_submission(preds=all_preds, dataset=my_dataset, document_ids=document_ids, id2label=id2label)
    if len(results) == 0:
        print("Error in create_submission(): No predictions made, probably because the model is not learning. Check the model and the data.")
        return
    df = pd.DataFrame(results)
    df=df[["row_id", "document", "token", "label"]]
    print(df)
    df.to_csv(filename, index=False)

create_submission(MyModel(parameter['model'], len(label2id)).to(device), "submission_just_dumb.csv")
#create_submission(learner.model, "submission.csv")


Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


still learning classifier.weight parameter_size: torch.Size([13, 768])
still learning classifier.bias parameter_size: torch.Size([13])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


      row_id  document  token             label
0          0         7      0  I-STREET_ADDRESS
1          1         7      1  I-STREET_ADDRESS
2          2         7      2  I-STREET_ADDRESS
3          3         7      3  I-STREET_ADDRESS
4          4         7      4  B-STREET_ADDRESS
...      ...       ...    ...               ...
7171    7171       123   1691  I-STREET_ADDRESS
7172    7172       123   1691           B-EMAIL
7173    7173       123   1691    I-URL_PERSONAL
7174    7174       123   1691       B-PHONE_NUM
7175    7175       123   1692  B-STREET_ADDRESS

[7176 rows x 4 columns]


In [11]:
data = json.load(open(train_path))
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "labels": [x["labels"] for x in data],
})
    
tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id, "max_length": parameter["max_length"], "all_labels_list": target}, num_proc=parameter["num_proc"])
ds=ds.filter(filter_no_pii, num_proc=parameter["num_proc"])


data_len=len(ds)
train_len=int(len(ds)*(1-parameter["train_test_split"]))
valid_len=len(ds)-train_len
train_data_idx=np.random.choice(data_len, train_len, replace=False)
valid_data_idx=np.array(list(set(range(data_len))-set(train_data_idx)))
print("train_len", train_len)
print("valid_len", valid_len)

# split ds in train and valid
train_ds=ds.select(train_data_idx)
valid_ds=ds.select(valid_data_idx)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map (num_proc=16):   0%|          | 0/6807 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/6807 [00:00<?, ? examples/s]

train_len 5445
valid_len 1362


In [12]:
# set environment variables: TOKENIZERS_PARALLELISM=false
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
tokenizer.add_tokens(AddedToken("\n", normalized=False))

train_dataset = PiiDataset(train_ds)
valid_dataset = PiiDataset(valid_ds)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=parameter['batch_size'], shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=parameter['inference_batch_size'], shuffle=False)
my_model=MyModel(parameter['model'], len(label2id))

learner=Learner(my_model, train_dataloader, valid_dataloader, parameter=parameter)
learner.get_accuracy()
if parameter["epochs_before_unfreeze"] > 0:
    learner.fit(lr=parameter['lr'], epochs=parameter["epochs_before_unfreeze"])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\neptune\common\warnings.py:71: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only b

still learning classifier.weight parameter_size: torch.Size([13, 768])
still learning classifier.bias parameter_size: torch.Size([13])
https://app.neptune.ai/bernd.heidemann/PII/e/PII-185


C:\Users\Bernd\AppData\Local\Temp\ipykernel_149980\157558940.py:120: RuntimeWarning: invalid value encountered in scalar divide
  f5 = (1 + 5**2) * (precision * recall) / ((5**2 * precision) + recall)


KeyboardInterrupt: 

In [ ]:
learner.model.unfreeze()

In [ ]:
for i in range(parameter["repeat_unfreeze_train_n_times"]):
    learner.fit(lr=parameter['lr']*parameter["lr_scale_unfreeze"], epochs=parameter["epochs_after_unfreeze"])
    create_submission(learner.model, f"submission_{i}.csv")

Training:   0%|          | 0/1164 [00:00<?, ?it/s]

In [ ]:
create_submission(learner.model, f"submission.csv")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


    row_id  document  token           label
0        0         7      6  B-NAME_STUDENT
1        1         7      9  B-NAME_STUDENT
2        2         7     10  I-NAME_STUDENT
3        3         7     52  B-NAME_STUDENT
4        4         7     53  I-NAME_STUDENT
5        5         7     55  B-NAME_STUDENT
6        6         7     56  I-NAME_STUDENT
7        7         7    479  B-NAME_STUDENT
8        8         7    482  B-NAME_STUDENT
9        9         7    483  I-NAME_STUDENT
10      10        10      0  B-NAME_STUDENT
11      11        10      1  I-NAME_STUDENT
12      12        10    464  B-NAME_STUDENT
13      13        10    465  I-NAME_STUDENT
14      14        16      4  B-NAME_STUDENT
15      15        16      5  I-NAME_STUDENT
16      16        20      5  B-NAME_STUDENT
17      17        20      6  I-NAME_STUDENT
18      18        20    328  B-NAME_STUDENT
19      19        20    330  B-NAME_STUDENT
20      20        56     12  B-NAME_STUDENT
21      21        56     13  I-N